In [ ]:
#imports
import json
import pickle
import pandas as pd

#file references
data_dump = "data/data_dump.json"

In [ ]:
############################################################
# read JSON file and save data into list of dicts
############################################################

json_file = open(data_dump)
json_str = json_file.read()
json_data = json.loads(json_str) # is list of dicts

# nbr of elements in data_dump 
print(len(json_data)) 
# data_dict = (json_data)[1] #dict

In [ ]:
############################################################
# reducing data size (making a copy of json_data)
# removes meta tags
# removes description -> tree
# removes entity -> ID
# removes entity -> value -> thesaurus entries
# removes entity -> rect -> coordinates 
############################################################

# keys that should remain in the dicts --> desc. (contains entities, relations & tree) 
keep = ["description"] # ["archivnr","material","kuenstler","bildthema","description"]  
                     

skinny_data = []                       
for diction in json_data:               
    skinny_dict = {k: diction[k] for k in keep if k in diction}
    #skinny_data.append(skinny_dict)


    print('removed: ',skinny_dict['description'].pop('tree', None))
    
    
    entities = skinny_dict['description']['entities']
    for x in enumerate(entities): #x gives index int
        
        index = ''
        #print('----')
        #print ('Orig. ',x[1]) #x[1] is the dict
        #print('vals ',x[1].values())
        #print('-')
        #print('removed: ', x[1].pop('id',None))
        x[1].pop('id',None)
        for y in enumerate(x[1].items()):
            temp_val = ''
            temp_key = ''
            #print(y[1])
            #print('+')
            if type(y[1][1]) == list:
                if type(y[1][1][0]) == dict:
                    temp_key = y[1][0]
                    #print('val: ',y[1][1][0]['value'])
                    if 'value' in y[1][1][0]:
                        temp_val = y[1][1][0]['value']
                        x[1][temp_key] = temp_val
                    else: 
                        print('no value key in: ', y[1][1][0])
                    if 'coordinates' in y[1][1][0]:
                        #print('removed: ', x[1].pop('coordinates',None))
                        x[1].pop('coordinates',None)
                
            #if type(y[1][1]) == list:
                #print('is list: ', y[1][1])
            #x[1][temp_key]
            #print(sorted(x[1].items()))



    skinny_data.append(skinny_dict)

In [ ]:
############################################################
# save list of dicts into JSON file
############################################################

# replaces special characters
# ä->ae | ö->oe | ü->ue | ß->sz

jsonList = json.dumps(skinny_data).replace("\\u00c3\\u00a4","ae").replace("\\u00c3\\u00bc","ue").replace("\\u00c3\\u0178","sz").replace("\\u00c3\\u201e","oe").replace("\\u00c3\\u00b6","oe").replace("\\u00c3\\u2013","oe").replace("\\u00c3\\u0153","ue").replace("\\u00c3\\u00a9","")


jsonFile = open("data/skinny_data.json", "w")
jsonFile.write(jsonList)
jsonFile.close()

In [ ]:
############################################################
# creating more dicts and saving into json
# Making Dataframes (columns: from(entity) - to(entity) - relation)
# Part 1
############################################################
data_processed = "data/skinny_data.json"
json_file = open(data_processed)
json_str = json_file.read()
json_data = json.loads(json_str) # is list of dicts

skinny_data = json_data
#new dicts that only contain relations/entities (easier to grab data)
relation_dict = []
entity_dict = []
for diction in skinny_data:
    for key, value in diction.items():
        for k, v in value.items():
            if k == 'entities':
                #print(v)
                #for x in v:
                    #print(x)
                entity_dict.append(v)
            if k == 'relations':
                #print(v)
                relation_dict.append(v)

entityList = json.dumps(entity_dict)
entityFile = open("data/entity_data.json", "w")
entityFile.write(entityList)
entityFile.close()

relList = json.dumps(relation_dict)
relFile = open("data/relation_data.json", "w")
relFile.write(relList)
relFile.close()

In [ ]:
############################################################
# Making Dataframes (columns: from(entity) - to(entity) - relation)
# Part 2
############################################################
entity_dict = json.loads(open("data/entity_data.json").read()) # load entity dictionaries
relation_dict = json.loads(open("data/relation_data.json").read()) # iload relation dictionaries

#### adding the archive number
archiv_list = []
for diction in json_data:
    archivnr = diction['archivnr']
    archiv_list.append(archivnr)
####

index = 0
df_list=[] #list of multiple KGs
list_of_lists = [] #list of KGs as lists rather than DFs
for diction in relation_dict:
    KnGr=[] # knowledge graph of image as a list
    KnGr.append(["Archivnummer",str(archiv_list[index]),"hat ID"])
    for value in diction:
        entity_from = entity_dict[index][value[0]] 
        entity_to = entity_dict[index][value[1]]
        relation = value[2]
        
        #print(type(entity_from))
        #print(entity_from)
        #temp_entity = entity(entity_from)
        #print(type(temp_entity.val))
        KnGr.append([str(entity_from),str(entity_to),relation])
        #print(KnGr[0])
        
    #print(KnGr)
    
    list_of_lists.append(KnGr)
    df=pd.DataFrame(KnGr,columns=['from','to','rel'])
    df.style
    df_list.append(df) #KnGr_list
    index +=1
#print(KnGr_list)


pickle.dump(df_list, open('data/all_Dataframes.pkl', 'wb')) # save all dataframes stored in a list
pickle.dump(list_of_lists, open('data/all_KGs_as_lists.pkl', 'wb')) # save all KGs in list-form stored in list
pickle.dump(archiv_list, open('data/archivlist.pkl','wb')) # just for double checking, the archive numbers

In [ ]:
entity_dict = json.loads(open("data/entity_data.json").read()) # load entity dictionaries
relation_dict = json.loads(open("data/relation_data.json").read()) # iload relation dictionaries

In [ ]:
############################################################
# Making Dataframes (columns: from(entity) - to(entity) - relation)
# Part 3 - altering the graphs to add more entities
############################################################



index = 0
all_dfs = [] # all dataframes as a list
#####
df = []
all_e_r_lists =[]
#####

for diction in relation_dict:
    
    ####
    dfs=[] # list of dataframes, not converted
    ####
    e_r_list = []
    e_r_list.append(["Archivnummer",str(archiv_list[index]),"hat ID"])
    for value in diction:
        entity_from = entity_dict[index][value[0]] 
        entity_to = entity_dict[index][value[1]]
        relation = value[2]
        ######################
        # this is where the entites should be split
        ######################
        
        e_r = [entity_from,entity_to,relation]
        e_r_list.append(e_r)

        #print(entity_from)
        if len(entity_from)>1:
            for i,(k,v) in enumerate(entity_from.items()):
                #print(i, v)
                e_r = [entity_from,v,k]
                #print(e_r)
                e_r_list.append(e_r)

        ####
    #print(e_r_list)   
    all_e_r_lists.append(e_r_list) 
    #print(all_e_r_lists)
    df = pd.DataFrame(e_r_list,columns=['from','to','rel'])  
    all_dfs.append(df) 
    #print(KnGr)
    #list_of_lists.append(KnGr)
    #all_dfs.append(dfs)
    #df=pd.DataFrame(KnGr,columns=['from','to','rel'])
    #df.style
    #df_list.append(df) #KnGr_list
    index +=1
#print(KnGr_list)


pickle.dump(all_dfs, open('data/all_Dataframes2.pkl', 'wb')) # save all dataframes stored in a list
pickle.dump(all_e_r_lists, open('data/all_KGs_as_lists2.pkl', 'wb')) # save all KGs in list-form stored in list

In [ ]:
############################################################
# removing the keys (just makes it easier to read)
# converting df entries to string
############################################################
#load the data
all_dfs = pickle.load(open('data/all_Dataframes2.pkl', 'rb'))
all_e_r_lists = pickle.load(open('data/all_KGs_as_lists2.pkl','rb'))

one_df = all_dfs[0]

one_df.style #testing that the df is correct


In [ ]:
pickle.dump(one_df,open('graphs/random/sample_df_altered.pkl','wb'))

In [ ]:
new_dfs=[]
all_rows= []

for df in all_dfs:
    rows=[]
    df_list = df.values.tolist()

    for val in df_list:
        #print(val)
        row = []
        
        for v in val:
            #print(v)
            if type(v)==dict: 
                my_list = []
                for x in v.values():
                    #print('x ', x)
                    my_list.append(x)
                    i = '_'.join(map(str, my_list))
                row.append(str(i))
            else:
                row.append(str(v))
        #print('list ',row)
        rows.append(row)
    new_df = pd.DataFrame(rows,columns=['from','to','rel']) 
    new_dfs.append(new_df)
    all_rows.append(rows)


In [ ]:
pickle.dump(new_dfs, open('data/all_Dataframes2.pkl', 'wb'))
pickle.dump(all_rows, open('data/all_rows.pkl','wb'))

In [ ]:
print(len(new_dfs))

new_dfs[5].style